## Level 1

In [16]:
import typing as tp


def solution(queries) -> tp.List[str]:
    doc_ = ""
    cursor = 0

    def append(snippet):
        nonlocal doc_, cursor

        doc_ += snippet
        cursor = len(doc_)
        return doc_

    def delete():
        nonlocal doc_, cursor

        doc_ = doc_[:cursor] + doc_[cursor + 1 :]
        return doc_

    def move(offset):
        nonlocal doc_, cursor

        offset = int(offset)
        cursor = offset
        # right = left + 1
        if cursor < 0:
            cursor = 0
        elif cursor > len(doc_):
            cursor = len(doc_)
        return doc_

    result = []
    for q in queries:
        operation = q[0]

        if operation == "APPEND":
            result.append(append(q[1]))
        elif operation == "DELETE":
            result.append(delete())
        elif operation == "MOVE":
            result.append(move(q[1]))

    return result


queries = [
    ["APPEND", "Hello"],
    ["APPEND", ","],
    ["APPEND", "World"],
    ["MOVE", "0"],
    ["DELETE"],
    ["DELETE"],
    ["MOVE", "20"],
    ["DELETE"],
]
result = solution(queries)
print(result)

['Hello', 'Hello,', 'Hello,World', 'Hello,World', 'ello,World', 'llo,World', 'llo,World', 'llo,World']


## Level 2

In [21]:
import typing as tp


def solution(queries) -> tp.List[str]:
    doc_ = ""
    cursor = 0
    selection = (-1, -1)
    clipboard = None

    def append(snippet):
        nonlocal doc_, cursor, selection

        left, right = selection
        if left != -1:
            doc_ = doc_[:left] + snippet + doc_[right:]
            cursor = left + len(snippet)
            selection = (-1, -1)
        else:
            doc_ += snippet
            cursor = len(doc_)
        return doc_

    def delete():
        nonlocal doc_, cursor, selection

        left, right = selection
        if left != -1:
            doc_ = doc_[:left] + doc_[right:]
            cursor = left
            selection = (-1, -1)
        else:
            doc_ = doc_[:cursor] + doc_[cursor + 1 :]
        return doc_

    def move(offset):
        nonlocal cursor, selection

        offset = int(offset)
        cursor = offset

        if cursor < 0:
            cursor = 0
        elif cursor > len(doc_):
            cursor = len(doc_)

        selection = (-1, -1)
        return doc_

    def select(start, end):
        nonlocal selection

        selection = (int(start), int(end))
        return doc_

    def cut():
        nonlocal doc_, cursor, selection, clipboard

        left, right = selection
        if left != -1:
            clipboard = doc_[left:right]
            doc_ = doc_[:left] + doc_[right:]
            cursor = left
            selection = (-1, -1)
        return doc_

    def paste():
        nonlocal doc_, cursor, clipboard

        if not clipboard:
            return doc_

        doc_ = doc_[:cursor] + clipboard + doc_[cursor:]
        cursor += len(clipboard)
        return doc_

    result = []
    for q in queries:
        operation = q[0]

        if operation == "APPEND":
            result.append(append(q[1]))
        elif operation == "DELETE":
            result.append(delete())
        elif operation == "MOVE":
            result.append(move(q[1]))
        elif operation == "SELECT":
            result.append(select(q[1], q[2]))
        elif operation == "CUT":
            result.append(cut())
        elif operation == "PASTE":
            result.append(paste())

    return result


queries = [
    ["APPEND", "Hello  World"],
    ["SELECT", "5", "7"],
    ["APPEND", ","],
    ["SELECT", "4", "6"],
    ["DELETE"],
    ["MOVE", "5"],
    ["APPEND", ", great"],
    ["SELECT", "4", "11"],
    ["CUT"],
    ["MOVE", "4"],
    ["PASTE"],
    ["PASTE"],
]
result = solution(queries)
print(result)

['Hello  World', 'Hello  World', 'Hello,World', 'Hello,World', 'HellWorld', 'HellWorld', 'HellWorld, great', 'HellWorld, great', 'Hellgreat', 'Hellgreat', 'HellWorld, great', 'HellWorld, World, great']


## Level 3

In [33]:
import typing as tp


def solution(queries) -> tp.List[str]:
    doc_ = ""
    cursor = 0
    selection = (-1, -1)
    clipboard = None
    history = {0: (doc_, cursor, selection, clipboard)}
    version = 0
    can_redo = False  # any modification to the document after undo should not be redo

    def append(snippet):
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        left, right = selection
        if left != -1:
            doc_ = doc_[:left] + snippet + doc_[right:]
            cursor = left + len(snippet)
            selection = (-1, -1)
        else:
            doc_ += snippet
            cursor = len(doc_)

        version += 1
        history[version] = (doc_, cursor, selection, clipboard)
        can_redo = False

        return doc_

    def delete():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        left, right = selection
        if left != -1:
            doc_ = doc_[:left] + doc_[right:]
            cursor = left
            selection = (-1, -1)
        else:
            doc_ = doc_[:cursor] + doc_[cursor + 1 :]

        version += 1
        history[version] = (doc_, cursor, selection, clipboard)
        can_redo = False
        return doc_

    def move(offset):
        nonlocal cursor, selection, version, can_redo

        offset = int(offset)
        cursor = offset

        if cursor < 0:
            cursor = 0
        elif cursor > len(doc_):
            cursor = len(doc_)

        selection = (-1, -1)

        version += 1
        history[version] = (doc_, cursor, selection, clipboard)
        return doc_

    def select(start, end):
        nonlocal selection, version, can_redo

        selection = (int(start), int(end))

        version += 1
        history[version] = (doc_, cursor, selection, clipboard)
        return doc_

    def cut():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        left, right = selection
        if left == -1:
            return doc_

        clipboard = doc_[left:right]
        doc_ = doc_[:left] + doc_[right:]
        cursor = left
        selection = (-1, -1)

        version += 1
        history[version] = (doc_, cursor, selection, clipboard)
        can_redo = False

        return doc_

    def paste():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        if not clipboard:
            return doc_

        doc_ = doc_[:cursor] + clipboard + doc_[cursor:]
        cursor += len(clipboard)

        version += 1
        history[version] = (doc_, cursor, selection, clipboard)
        can_redo = False

        return doc_

    def undo():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        if version - 1 not in history:
            return doc_

        version -= 1
        doc_, cursor, selection, clipboard = history[version]
        can_redo = True
        return doc_

    def redo():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        if not can_redo:
            # operation is ignored
            return doc_

        if version + 1 not in history:
            return doc_

        version += 1
        doc_, cursor, selection, clipboard = history[version]
        return doc_

    result = []
    for q in queries:
        operation = q[0]

        if operation == "APPEND":
            result.append(append(q[1]))
        elif operation == "DELETE":
            result.append(delete())
        elif operation == "MOVE":
            result.append(move(q[1]))
        elif operation == "SELECT":
            result.append(select(q[1], q[2]))
        elif operation == "CUT":
            result.append(cut())
        elif operation == "PASTE":
            result.append(paste())
        elif operation == "UNDO":
            result.append(undo())
        elif operation == "REDO":
            result.append(redo())

    return result


queries = [
    ["APPEND", "Hello  World"],
    ["UNDO"],
    ["REDO"],
    ["REDO"],
    ["SELECT", "5", "7"],
    ["APPEND", ","],
    ["UNDO"],
    ["APPEND", ":"],
    ["REDO"],
    ["UNDO"],
    ["UNDO"],
    ["UNDO"],
    ["UNDO"],
]
result = solution(queries)
print(result)

['Hello  World', '', 'Hello  World', 'Hello  World', 'Hello  World', 'Hello,World', 'Hello  World', 'Hello:World', 'Hello:World', 'Hello  World', 'Hello  World', '', '']


## Level 4

In [39]:
import typing as tp


def solution(queries) -> tp.List[str]:
    documents = {}
    curr = None
    clipboard = None

    doc_ = ""
    cursor = 0
    selection = (-1, -1)
    history = {0: (doc_, cursor, selection)}
    version = 0
    can_redo = False

    def append(snippet):
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        left, right = selection
        if left != -1:
            doc_ = doc_[:left] + snippet + doc_[right:]
            cursor = left + len(snippet)
            selection = (-1, -1)
        else:
            doc_ += snippet
            cursor = len(doc_)

        version += 1
        history[version] = (doc_, cursor, selection)
        can_redo = False

        return doc_

    def delete():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        left, right = selection
        if left != -1:
            doc_ = doc_[:left] + doc_[right:]
            cursor = left
            selection = (-1, -1)
        else:
            doc_ = doc_[:cursor] + doc_[cursor + 1 :]

        version += 1
        history[version] = (doc_, cursor, selection)
        can_redo = False
        return doc_

    def move(pos):
        nonlocal cursor, selection, version, can_redo

        pos = int(pos)
        cursor = pos

        if cursor < 0:
            cursor = 0
        elif cursor > len(doc_):
            cursor = len(doc_)

        selection = (-1, -1)

        version += 1
        history[version] = (doc_, cursor, selection)
        return doc_

    def select(start, end):
        nonlocal selection, version, can_redo

        selection = (int(start), int(end))

        version += 1
        history[version] = (doc_, cursor, selection)
        return doc_

    def cut():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        left, right = selection
        if left == -1:
            return doc_

        clipboard = doc_[left:right]
        doc_ = doc_[:left] + doc_[right:]
        cursor = left
        selection = (-1, -1)

        version += 1
        history[version] = (doc_, cursor, selection)
        can_redo = False

        return doc_

    def paste():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo

        if not clipboard:
            return doc_

        doc_ = doc_[:cursor] + clipboard + doc_[cursor:]
        cursor += len(clipboard)

        version += 1
        history[version] = (doc_, cursor, selection)
        can_redo = False

        return doc_

    def undo():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo, history

        if version - 1 not in history:
            return doc_

        version -= 1
        doc_, cursor, selection = history[version]
        can_redo = True
        return doc_

    def redo():
        nonlocal doc_, cursor, selection, clipboard, version, can_redo, history

        if not can_redo:
            # operation is ignored
            return doc_

        if version + 1 not in history:
            return doc_

        version += 1
        doc_, cursor, selection = history[version]
        return doc_

    def create(name):
        nonlocal curr, documents, doc_, cursor, selection, can_redo, history, version

        save_document()

        if name in documents:
            curr = name
            doc_, cursor, selection, history, version, can_redo = documents[curr]
            return doc_
        else:
            curr = name
            doc_, cursor, selection, history, version, can_redo = (
                "",
                0,
                (-1, -1),
                {0: ("", 0, (-1, -1))},
                0,
                False,
            )
            documents[curr] = (doc_, cursor, selection, history, version, can_redo)
            return doc_

    def switch(name):
        nonlocal curr, documents, doc_, cursor, selection, can_redo, history, version

        save_document()

        if name not in documents:
            return ""
        else:
            curr = name
            doc_, cursor, selection, history, version, can_redo = documents[curr]
            return doc_

    def save_document():
        nonlocal curr, documents, doc_, cursor, selection, can_redo, history, version

        if curr:
            documents[curr] = (doc_, cursor, selection, history, version, can_redo)

    result = []
    for q in queries:
        operation = q[0]

        if operation == "APPEND":
            result.append(append(q[1]))
        elif operation == "DELETE":
            result.append(delete())
        elif operation == "MOVE":
            result.append(move(q[1]))
        elif operation == "SELECT":
            result.append(select(q[1], q[2]))
        elif operation == "CUT":
            result.append(cut())
        elif operation == "PASTE":
            result.append(paste())
        elif operation == "UNDO":
            result.append(undo())
        elif operation == "REDO":
            result.append(redo())
        elif operation == "CREATE":
            result.append(create(q[1]))
        elif operation == "SWITCH":
            result.append(switch(q[1]))

    return result


queries = [
    ["CREATE", "document1"],
    ["CREATE", "document2"],
    ["CREATE", "document1"],
    ["SWITCH", "document1"],
    ["APPEND", "Hello  World!"],
    ["UNDO"],
    ["REDO"],
    ["SELECT", "7", "12"],
    ["CUT"],
    ["SWITCH", "document3"],
    ["SWITCH", "document2"],
    ["PASTE"],
    ["SWITCH", "document1"],
    ["DELETE"],
]
result = solution(queries)
print(result)

['', '', '', '', 'Hello  World!', '', 'Hello  World!', 'Hello  World!', 'Hello  !', '', '', 'World', 'Hello  !', 'Hello  ']
